# Load Variables

In [1]:
%run "scripts/load_daily_variables.py"

Loading Variables: 100%|██████████| 26/26 [02:06<00:00,  4.87s/it]


# High Beta with Recent Elevated Volatility and Activity

In [2]:
hbeta_vol_regime_5day = [sym for sym in symbols if 
                    (sym in results_finvizsearch.loc[results_finvizsearch['Beta'] > 2]['Ticker'].to_list()) and 
                    (symbols[sym].ATRs_Traded.replace(np.inf, np.nan).dropna().iloc[-5:] >= 1.5).any() and
                    (symbols[sym].RVol.replace(np.inf, np.nan).dropna().iloc[-5:] >= 1.5).any()
                    ]
hbeta_vol_regime_20day = [sym for sym in symbols if 
                    (sym in results_finvizsearch.loc[results_finvizsearch['Beta'] > 2]['Ticker'].to_list()) and 
                    (symbols[sym].ATRs_Traded.replace(np.inf, np.nan).dropna().iloc[-20:] >= 1.5).any() and
                    (symbols[sym].RVol.replace(np.inf, np.nan).dropna().iloc[-20:] >= 1.5).any() and
                    (sym not in hbeta_vol_regime_5day)
                    ]

- Quant Rating of stocks with elevated volatility in the last 5 days

In [3]:
mask = daily_quant_rating_df.index.isin(hbeta_vol_regime_5day)
col = daily_quant_rating_df.columns[-2]
(
daily_quant_rating_df.loc[mask, col]
.sort_values(ascending=False)
.head(50)
)

Symbol
LASR    4.979624
GRAL    4.963950
CRDO    4.926332
CYH     4.913793
W       4.912226
EBS     4.744514
QUBT    4.466480
TXG     4.393855
BBAI    4.391061
DAN     3.930168
PACB    3.896648
AMRC    3.631285
ONDS    3.497277
LMND    3.488768
IREN    3.477876
AMPX    3.452349
ASTS    3.452008
CIFR    3.430565
LC      3.428523
CTMX    3.427161
RKT     3.368958
DPRO    3.277400
NVAX    3.219877
SPIR    3.158271
QS      3.100408
AOSL    3.082029
LIF     3.074541
RIOT    3.065351
CABA    3.056161
AEHR    3.045269
U       3.025528
NCLH    3.024166
RUN     3.001021
SHOP    2.961538
RDW     2.942818
ARM     2.871681
JANX    2.836964
TREE    2.826413
APPS    2.820626
NTLA    2.809735
KOPN    2.789653
SLDB    2.750170
CG      2.733833
NNE     2.699455
RYAM    2.675630
OCGN    2.657931
CZR     2.642954
MEG     2.636828
BEAM    2.632403
IONQ    2.606875
Name: 2026-01-23, dtype: float64

- Quant Rating of stocks with elevated volatility in the last 20 days minus those in hbeta_vol_regime_5day

In [4]:
mask = daily_quant_rating_df.index.isin(hbeta_vol_regime_20day)
col = daily_quant_rating_df.columns[-2]
(
daily_quant_rating_df.loc[mask, col]
.sort_values(ascending=False)
.head(50)
)

Symbol
CTRN    4.791536
SEZL    4.789969
FLNC    4.761755
GAP     4.727273
PSIX    4.703762
AS      4.648903
GLXY    4.625392
DAVE    4.603448
APP     4.547022
NVTS    4.474860
CDNA    4.413408
BFLY    4.360335
INGM    4.117318
SFIX    3.918994
INOD    3.871508
LSPD    3.782123
VIK     3.723464
SKYT    3.497617
SLS     3.495235
BE      3.494554
COMP    3.490196
PRAX    3.490129
REAL    3.462560
APLD    3.461879
SOFI    3.437713
SMTC    3.426140
ADPT    3.423758
RDDT    3.390061
HOOD    3.387679
TMDX    3.367257
NXT     3.323009
JMIA    3.321647
CENX    3.303608
BTBT    3.282845
Z       3.245098
NBIS    3.241661
ACHR    3.217835
AFRM    3.209666
BKSY    3.207965
AMSC    3.199455
UMAC    3.186521
NAGE    3.155548
AUR     3.124575
TPC     3.090197
SAIA    3.086453
ODD     3.046971
FULC    3.037781
OMER    3.027570
QXO     3.020762
PTON    2.984343
Name: 2026-01-23, dtype: float64

# YTD Performance vs. Quant Rating

In [5]:
import plotly.express as px
quant_col = daily_quant_rating_df.columns[-2]
quant_rating = daily_quant_rating_df[quant_col].reset_index().rename(columns={quant_col:'Quant Rating'})
perf = results_finvizsearch[['Ticker', 'Performance (YearToDate)']]
df = quant_rating.merge(perf, left_on='Symbol', right_on='Ticker', how='inner')
# df = df.loc[df.Ticker.isin(fu.sector_industry_member_search('Basic Materials', level='sector'))]
# Assign color based on logic:
# quant rating > 4.8 & Performance < 50 -> green
# quant rating < 3 & Performance > 50 -> red
# else -> gray
def assign_color(row):
    if (row['Quant Rating'] > 4.8) and (row['Performance (YearToDate)'] < 50):
        return 'green'
    elif (row['Quant Rating'] < 3) and (row['Performance (YearToDate)'] > 50):
        return 'red'
    elif (row['Quant Rating'] > 2.5) and (row['Quant Rating'] < 4) and (row['Performance (YearToDate)'] > 50):
        return 'yellow'
    else:
        return 'gray'

df['Color'] = df.apply(assign_color, axis=1)

px.scatter(
    df.assign(
        Color_Label=df['Color'].map({'gray': 'uncertain', 'green': 'undervalued', 'red': 'overvalued', 'yellow': 'worth watching'})
    ),
    x='Performance (YearToDate)',
    y='Quant Rating',
    hover_data=['Ticker'],
    color='Color_Label',
    color_discrete_map={'uncertain': 'gray', 'undervalued':'green', 'overvalued':'red', 'worth_watching': 'yellow'},
    category_orders={'Color_Label': ['uncertain', 'undervalued', 'overvalued', 'worth_watching']},
    labels={'Color_Label': 'Category'},
    title='YTD Performance vs. Quant Rating'
)

# Interest List Long Performance vs. Quant Rating

- Quant Rating for symbol in interest_list_long

In [6]:
mask = daily_quant_rating_df.index.isin(interest_list_long)
col = daily_quant_rating_df.columns[-2]
quantdf_illong = (
daily_quant_rating_df.loc[mask, col]
.sort_values(ascending=False)
)
quantdf_illong.head(50)

Symbol
MU      4.998433
NGD     4.993730
IAG     4.990596
TTMI    4.981191
LASR    4.979624
NUTX    4.976489
TSM     4.973354
NEM     4.970219
CDE     4.968652
GOLD    4.967085
GRAL    4.963950
CGAU    4.962382
GFI     4.957680
GM      4.954545
CSTM    4.949843
CIEN    4.948276
COHR    4.940439
KGC     4.937304
INDV    4.935737
INTC    4.934169
VSAT    4.932602
AMKR    4.931034
WLDN    4.929467
CRDO    4.926332
AU      4.924765
MFC     4.923197
DB      4.921630
LQDA    4.915361
CYH     4.913793
W       4.912226
PFE     4.910658
DRH     4.906250
INCY    4.904389
ABEV    4.902821
NBR     4.899687
PAAS    4.898119
HBM     4.896552
AEM     4.874608
RERE    4.866771
CIB     4.858934
OI      4.854232
GH      4.851097
KALU    4.832288
RIGL    4.826019
LCII    4.822884
EGO     4.818182
ENVA    4.813480
EQX     4.799373
BVN     4.794671
KRYS    4.793103
Name: 2026-01-23, dtype: float64

- Quant Rating vs. YTD Performance for symbols in interest_list_long

In [7]:
import plotly.express as px
quant_col = daily_quant_rating_df.columns[-2]
quant_rating = quantdf_illong.reset_index().rename(columns={col:'Quant Rating'})
perf = results_finvizsearch[['Ticker', 'Performance (YearToDate)']]
df = quant_rating.merge(perf, left_on='Symbol', right_on='Ticker', how='inner')
# df = df.loc[df.Ticker.isin(fu.sector_industry_member_search('Basic Materials', level='sector'))]
# Assign color based on logic:
# quant rating > 4.8 & Performance < 50 -> green
# quant rating < 3 & Performance > 50 -> red
# else -> gray
def assign_color(row):
    if (row['Quant Rating'] > 4.8) and (row['Performance (YearToDate)'] < 50):
        return 'green'
    elif (row['Quant Rating'] < 3) and (row['Performance (YearToDate)'] > 50):
        return 'red'
    elif (row['Quant Rating'] > 2.5) and (row['Quant Rating'] < 4) and (row['Performance (YearToDate)'] > 50):
        return 'yellow'
    else:
        return 'gray'

df['Color'] = df.apply(assign_color, axis=1)

px.scatter(
    df.assign(
        Color_Label=df['Color'].map({'gray': 'uncertain', 'green': 'undervalued', 'red': 'overvalued', 'yellow': 'worth watching'})
    ),
    x='Performance (YearToDate)',
    y='Quant Rating',
    hover_data=['Ticker'],
    color='Color_Label',
    color_discrete_map={'uncertain': 'gray', 'undervalued':'green', 'overvalued':'red', 'worth_watching': 'yellow'},
    category_orders={'Color_Label': ['uncertain', 'undervalued', 'overvalued', 'worth_watching']},
    labels={'Color_Label': 'Category'},
    title='YTD Performance vs. Quant Rating'
)

# INTEREST LIST LONG FROM FUNDAMENTALS

- Sorted by Quant Rating

In [8]:
interest_list_fundamental = [
    sym for sym in symbols if 
    (symbols[sym].interest_factor is not None) and
    ('Fundamental' in symbols[sym].interest_factor)
]
col = daily_quant_rating_df.columns[-2]
quantdf_ifund = (
daily_quant_rating_df.loc[daily_quant_rating_df.index.isin(interest_list_fundamental), col]
.sort_values(ascending=False)
)
quantdf_ifund.head(50)

Symbol
MU      4.998433
NGD     4.993730
IAG     4.990596
TTMI    4.981191
LASR    4.979624
NUTX    4.976489
TSM     4.973354
NEM     4.970219
CDE     4.968652
GOLD    4.967085
GRAL    4.963950
CGAU    4.962382
GFI     4.957680
GM      4.954545
CSTM    4.949843
CIEN    4.948276
COHR    4.940439
KGC     4.937304
INDV    4.935737
INTC    4.934169
VSAT    4.932602
AMKR    4.931034
WLDN    4.929467
CRDO    4.926332
AU      4.924765
MFC     4.923197
DB      4.921630
LQDA    4.915361
CYH     4.913793
W       4.912226
PFE     4.910658
DRH     4.906250
INCY    4.904389
ABEV    4.902821
Name: 2026-01-23, dtype: float64

- Sorted by Relative Volume

In [9]:
sorted(
    {sym: symbols[sym].RVol.iloc[-1].item() for sym in interest_list_fundamental}.items(),
    key=lambda x: x[1],
    reverse=True
)

[('INTC', 2.42091629709158),
 ('LASR', 1.682624573404626),
 ('GRAL', 1.6214094751054444),
 ('AU', 1.526194074156647),
 ('CYH', 1.3312835335726287),
 ('NGD', 1.3275622862073326),
 ('DRH', 1.2596340267019595),
 ('INCY', 1.235616073221321),
 ('IAG', 1.1882965762194335),
 ('CDE', 1.1460152478999337),
 ('ABEV', 1.1370250142588658),
 ('GFI', 1.0965325086113578),
 ('MU', 1.0879519404458973),
 ('WLDN', 1.0760220986907214),
 ('PFE', 1.050555543888958),
 ('NEM', 0.98705755686909),
 ('CGAU', 0.9581379658499116),
 ('TTMI', 0.9553294683835553),
 ('GM', 0.9449269650359653),
 ('KGC', 0.9399209935217554),
 ('TSM', 0.9332741324669697),
 ('LQDA', 0.9288786432546088),
 ('DB', 0.9006396003174502),
 ('GOLD', 0.8789106640927741),
 ('NUTX', 0.8696765349651495),
 ('CIEN', 0.8182688973897091),
 ('CRDO', 0.8032771127465128),
 ('W', 0.7979515548840338),
 ('AMKR', 0.7524048309566477),
 ('VSAT', 0.6916536646712961),
 ('MFC', 0.6709388020884823),
 ('COHR', 0.6617824243415872),
 ('INDV', 0.6504984705116451),
 ('CSTM

- Quant Rating vs. YTD Performance for symbols with fundamental factor in interest_list_long

In [10]:
import plotly.express as px
quant_col = daily_quant_rating_df.columns[-2]
quant_rating = quantdf_ifund.reset_index().rename(columns={col:'Quant Rating'})
perf = results_finvizsearch[['Ticker', 'Performance (YearToDate)']]
df = quant_rating.merge(perf, left_on='Symbol', right_on='Ticker', how='inner')

px.scatter(
    df,
    x='Performance (YearToDate)',
    y='Quant Rating',
    hover_data=['Ticker'],
    title='YTD Performance vs. Quant Rating'
)